# LangGraph Music Router with Azure AI Telemetry

Welcome! This notebook demonstrates how to stream a LangGraph workflow that routes tool calls while capturing GenAI-compliant spans.

## 🎯 What You'll Build

By the end of this notebook, you'll have:
- ✅ Built a LangGraph workflow with tool routing
- ✅ Instrumented with Azure AI OpenTelemetry tracer
- ✅ Captured GenAI-compliant spans for model invocations and tool executions
- ✅ Streamed workflow execution with telemetry

## 💡 What You'll Learn

- How to instrument LangGraph workflows with OpenTelemetry
- How to use `langchain-azure-ai[opentelemetry]` with LangGraph
- How to capture telemetry for streaming workflows
- Best practices for observing multi-step graph workflows

> **Note**: This uses a music router scenario to demonstrate how OpenTelemetry patterns work with LangGraph's stateful agent architecture.

Ready to instrument LangGraph? Let's get started! 🚀

---

## Requirements
Install these packages before running the notebook.

```bash
pip install langchain langgraph langchain-openai "langchain-azure-ai[opentelemetry]" python-dotenv
```

Export the following environment variables (values may be blank if not used in this sample):
- `AZURE_OPENAI_API_KEY`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_VERSION`
- `AZURE_OPENAI_MODEL_NAME`
- `AZURE_AI_FOUNDRY_NAME`
- `AZURE_AOAI_ACCOUNT`
- `AZURE_SUBSCRIPTION_ID`
- `AZURE_RESOURCE_GROUP`
- `AZURE_AISEARCH_ENDPOINT`
- `AZURE_AISEARCH_INDEX`
- `AZURE_AISEARCH_RESOURCE_GROUP`
- `APPLICATION_INSIGHTS_CONNECTION_STRING`

The observability labs only reference this list of variables.


### Step 1: Import LangGraph, LangChain, and telemetry helpers
Bring in the required libraries and load environment variables for your chosen provider.


In [ ]:
from __future__ import annotations

import os

from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode

from langchain_azure_ai.callbacks.tracers import AzureAIOpenTelemetryTracer

load_dotenv(override=True)

MODEL_NAME = os.environ.get("AZURE_OPENAI_MODEL_NAME") or "gpt-4o-mini"


### Step 2: Configure the chat model and tracer
Initialise the chat client (GitHub Models or Azure OpenAI) and the Azure AI tracer so spans are emitted for every model call.


In [ ]:
TRACER = AzureAIOpenTelemetryTracer(
    connection_string=os.environ.get("APPLICATION_INSIGHTS_CONNECTION_STRING"),
    enable_content_recording=True,
    name="Music Router Agent",
)

MODEL = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-05-01-preview"),
    azure_deployment=MODEL_NAME,
)
print("Model ready with model:", MODEL_NAME)


### Step 3: Declare tools and bind them to the model
Define the music playback tools and bind them to the chat model so LangGraph can leverage them during execution.


In [ ]:
@tool
def play_song_on_spotify(song: str) -> str:
    """Simulated Spotify playback."""

    return f"Successfully played {song} on Spotify!"


@tool
def play_song_on_apple(song: str) -> str:
    """Simulated Apple Music playback."""

    return f"Successfully played {song} on Apple Music!"


TOOLS = [play_song_on_spotify, play_song_on_apple]
TOOL_NODE = ToolNode(TOOLS)
MODEL_WITH_TOOLS = MODEL.bind_tools(TOOLS, parallel_tool_calls=False)


### Step 4: Build the LangGraph workflow
Create the agent and tool nodes, wire up conditional edges, and compile the workflow with an in-memory checkpointer.


In [ ]:
def should_continue(state: MessagesState) -> str:
    messages = state["messages"]
    last_message = messages[-1]
    return "end" if not last_message.tool_calls else "continue"


def call_model(state: MessagesState) -> dict:
    messages = state["messages"]
    response = MODEL_WITH_TOOLS.invoke(messages)
    return {"messages": [response]}


WORKFLOW = StateGraph(MessagesState)
WORKFLOW.add_node("agent", call_model)
WORKFLOW.add_node("action", TOOL_NODE)
WORKFLOW.add_edge(START, "agent")
WORKFLOW.add_conditional_edges("agent", should_continue, {"continue": "action", "end": END})
WORKFLOW.add_edge("action", "agent")

MEMORY = MemorySaver()
APP = WORKFLOW.compile(checkpointer=MEMORY)
print("Workflow compiled.")

### Step 5: Stream an interaction
Execute the graph with the tracer callback to observe emitted spans for the model and tool steps in real time.


In [ ]:
config = {"configurable": {"thread_id": "1"}, "callbacks": [TRACER]}
input_message = HumanMessage(content="Can you play Taylor Swift's most popular song?")

for event in APP.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()